In [53]:
import fastf1, datetime
import pandas as pd
from standings import get_driver_points, get_constructor_points

fastf1.Cache.enable_cache("cache")
current_datetime = datetime.datetime.now()
points = {1: 25, 2: 18, 3: 15, 4: 12, 5: 10, 6: 8, 7: 6, 8: 4, 9: 2, 10: 1}

In [54]:
def get_safety_car_laps(race_status):
    count = 0
    for status in race_status:
        if '4' in str(status) or '5' in str(status) or '6' in str(status) or '7' in str(status):
            count += 1
    return count

In [58]:
from track_data import track_data, alternate_names

races = pd.DataFrame(columns=[
    'Driver Points', 'Team Points', 'GridPosition', 'FinishPosition', 'PitStops', 'Stint1Length', 'Stint1Compound', 'Stint2Length', 'Stint2Compound', 'Stint3Length', 'Stint3Compound', 'Stint4Length', 'Stint4Compound', 'Laps', 'LengthKM', 'TurnCount', 'DrsZones', 'PitLaneLengthM', 'DegradationLevel', 'StreetCircuit', 'AvgTrackTemp', 'AvgAirTemp', 'AvgHumidity', 'RainfallMM', 'SafetyCarLaps', 'Score'
])

for year in range(2018, 2019):#current_datetime.year + 1):
    schedule = fastf1.get_event_schedule(year)
    for _, event in schedule.iterrows():
        # If event format is not pre-season testing
        if event['EventFormat'] == 'testing':
            continue
        try:
            session = fastf1.get_session(year, event['EventName'], 'Race')
            session.load(weather=True)

            weather = session.weather_data
            avg_track_temp = weather['TrackTemp'].mean()
            avg_air_temp = weather['AirTemp'].mean()
            avg_humidity = weather['Humidity'].mean()
            rain = weather['Rainfall'].mean()

            laps = session.laps
            race_status = laps.groupby('LapNumber')['TrackStatus'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
            safety_car_laps = get_safety_car_laps(race_status)

            qualifying_results = session.results[['Abbreviation', 'GridPosition']]
            race_results = session.results[['Abbreviation', 'Position']]

            stint_data = (
                laps.groupby(["Driver", "Stint", "Compound"])
                .agg(stint_length=("LapNumber", "count"))
                .reset_index()
            )

            track_info = track_data.get(event['Location'])
            if track_info is None:
                for track_list in alternate_names:
                    if event['Location'] in track_list:
                        track_info = track_data.get(track_list[0])
                        break

            statuses = session.results['Status'].fillna('').astype(str)
            dnf_mask = (statuses != 'Finished') & (~statuses.str.startswith('+'))
            dnf_drivers = session.results[dnf_mask]['Abbreviation'].tolist()

            for _, driver in session.results.iterrows():
                if driver['Abbreviation'] in dnf_drivers:
                    continue
                driver_stints = stint_data[stint_data['Driver'] == driver['Abbreviation']]
                stint_lengths = driver_stints['stint_length'].tolist()
                stint_compounds = driver_stints['Compound'].tolist()

                row = {
                    'Driver Points': get_driver_points(year, event['RoundNumber'], driver['Abbreviation']),
                    'Team Points': get_constructor_points(year, event['RoundNumber'], driver['TeamName']),
                    'GridPosition': driver['GridPosition'],
                    'FinishPosition': driver['Position'],
                    'PitStops': len(driver_stints) - 1,
                    'Laps': track_info['laps'],
                    'LengthKM': track_info['length_km'],
                    'TurnCount': track_info['turn_count'],
                    'DrsZones': track_info['drs_zones'],
                    'PitLaneLengthM': track_info['pit_lane_length_m'],
                    'DegradationLevel': track_info['degradation_level'],
                    'StreetCircuit': track_info['street_circuit'],
                    'AvgTrackTemp': avg_track_temp,
                    'AvgAirTemp': avg_air_temp,
                    'AvgHumidity': avg_humidity,
                    'RainfallMM': rain,
                    'SafetyCarLaps': safety_car_laps
                }

                for i in range(4):
                    if i < len(stint_lengths):
                        row[f'Stint{i+1}Length'] = stint_lengths[i]
                        row[f'Stint{i+1}Compound'] = stint_compounds[i]
                    else:
                        row[f'Stint{i+1}Length'] = 0
                        row[f'Stint{i+1}Compound'] = None

                # Calculate score based on 2 * finish position points minus projected grid position points
                finish_points = points.get(driver['Position'], 0)
                grid_points = points.get(driver['GridPosition'], 0)
                row['Score'] = 2 * finish_points - grid_points

                races.loc[len(races)] = row
            
        except Exception as e:
            print("ERROR:", event['Location'], year, e)
            continue

core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '28'
req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
core        WARNING 	Car position data is unavailable!
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 5 complet

In [70]:
import fastf1.plotting
fastf1.plotting.list_compounds(fastf1.get_session(2018, 'Australia', 'Race'))

['HYPERSOFT',
 'ULTRASOFT',
 'SUPERSOFT',
 'SOFT',
 'MEDIUM',
 'HARD',
 'SUPERHARD',
 'INTERMEDIATE',
 'WET',
 'UNKNOWN',
 'TEST-UNKNOWN']

In [59]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
races

,Driver Points,Team Points,GridPosition,FinishPosition,PitStops,Stint1Length,Stint1Compound,Stint2Length,Stint2Compound,Stint3Length,Stint3Compound,Stint4Length,Stint4Compound,Laps,LengthKM,TurnCount,DrsZones,PitLaneLengthM,DegradationLevel,StreetCircuit,AvgTrackTemp,AvgAirTemp,AvgHumidity,RainfallMM,SafetyCarLaps,Score
0,25.0,40.0,3.0,1.0,1,25,ULTRASOFT,32,SOFT,0,None,0,None,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,35
1,18.0,22.0,1.0,2.0,1,17,ULTRASOFT,39,SOFT,0,None,0,None,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,11
2,15.0,40.0,2.0,3.0,1,17,ULTRASOFT,40,SOFT,0,None,0,None,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,12
3,12.0,20.0,8.0,4.0,1,25,SUPERSOFT,32,SOFT,0,None,0,None,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,20
4,10.0,12.0,10.0,5.0,1,25,ULTRASOFT,32,SOFT,0,None,0,None,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,19
5,8.0,20.0,4.0,6.0,1,20,SUPERSOFT,37,SOFT,0,None,0,None,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,4
6,6.0,7.0,7.0,7.0,1,23,ULTRASOFT,34,SOFT,0,None,0,None,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,6
7,4.0,22.0,15.0,8.0,1,24,ULTRASOFT,33,SUPERSOFT,0,None,0,None,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,8
8,2.0,12.0,11.0,9.0,1,24,ULTRASOFT,33,SUPERSOFT,0,None,0,None,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,4
9,1.0,7.0,9.0,10.0,1,21,ULTRASOFT,36,SOFT,0,None,0,None,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,0
